# HealthVision: AI Fall Detection Training

This notebook trains a Hybrid LSTM model on pose keypoints extracted from video to detect falls.
**Architecture**: Input (30 frames, 34 keypoints) -> LSTM/Transformer -> Class Probability

In [ ]:
!pip install kagglehub ultralytics
import kagglehub
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

# Import our local modules
from model import build_fall_detection_model, build_transformer_model

## 1. Download & Organize Datasets
We will auto-download the datasets and organize them into a standard structure.

In [ ]:
print("Downloading datasets...")
ur_fall_path = kagglehub.dataset_download("shahliza27/ur-fall-detection-dataset")

print("UR Fall Path:", ur_fall_path)

# Setup Directories
base_dir = "dataset_organized"
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)
os.makedirs(f"{base_dir}/fall", exist_ok=True)
os.makedirs(f"{base_dir}/normal", exist_ok=True)

def organize_dataset_leaves(source_dir, dest_base):
    count_fall = 0
    count_normal = 0
    
    # Walk deeply
    for root, dirs, files in os.walk(source_dir):
        # Check if LEAF node (has images)
        has_images = any(f.lower().endswith(('.png', '.jpg')) for f in files)
        
        if has_images:
            dirname = os.path.basename(root)
            lower = dirname.lower()
            
            target_class = None
            if 'adl' in lower or 'd0' in lower:
                target_class = 'normal'
            elif 'fall' in lower or 'f0' in lower:
                target_class = 'fall'
            
            if target_class:
                src = root
                dst = os.path.join(dest_base, target_class, dirname)
                # Copy this leaf folder
                if not os.path.exists(dst):
                    shutil.copytree(src, dst)
                    if target_class == 'normal': count_normal += 1
                    else: count_fall += 1
                    
    return count_fall, count_normal

print("Organizing files...")
f, n = organize_dataset_leaves(ur_fall_path, base_dir)
print(f"Organized: {f} Fall sequences, {n} Normal sequences.")

## 2. Preprocessing
Extract keypoints.

In [ ]:
!python preprocess.py

In [ ]:
# Load processed data
if os.path.exists('processed_data.npy'):
    data = np.load('processed_data.npy', allow_pickle=True).item()
    X = data['X']
    y = data['y']

    print(f"Total Samples: {X.shape[0]}")
    print(f"Feature Shape: {X.shape[1:]}") 
    
    unique_classes = np.unique(y)
    print(f"Classes found: {unique_classes}")
    num_classes = 3 # Force 3 classes architecture

    y_encoded = keras.utils.to_categorical(y, num_classes=num_classes)

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y)

    print(f"Training samples: {len(X_train)}")
    print(f"Testing samples: {len(X_test)}")

## 3. Train Model

In [ ]:
if 'X_train' in locals() and len(X_train) > 0:
    input_shape = (30, 34)
    model = build_fall_detection_model(input_shape, num_classes)
    model.summary()

    # Compute weights
    y_integers = np.argmax(y_encoded, axis=1)
    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_integers), y=y_integers)
    class_weight_dict = dict(zip(np.unique(y_integers), class_weights))
    print("Class Weights:", class_weight_dict)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=32,
        # class_weight=class_weight_dict, # Optional, use if imbalance is huge
        callbacks=[
            keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
        ]
    )
else:
    print("Skipping training, no data loaded.")

## 4. Evaluation & Export

In [ ]:
if 'model' in locals() and 'X_test' in locals():
    loss, acc = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {acc*100:.2f}%")

    # Export to TFLite with SELECT_TF_OPS for LSTM support
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
    # CRITICAL FIX for LSTM layers conversion:
    # Enable standard TFLite ops and specific TensorFlow ops (Select TF Ops)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS, 
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    # Disable experimental LOWERING of Tensor List ops which causes the 'legalize' error
    converter._experimental_lower_tensor_list_ops = False
    
    tflite_model = converter.convert()

    with open('fall_detection_model.tflite', 'wb') as f:
        f.write(tflite_model)
        
    print("SUCCESS: Model saved as 'fall_detection_model.tflite'")